0. Utils

In [ ]:
from logger import get_logger
from models.chats import ChatMessage
from models.settings import common_dependencies

logger = get_logger(__name__)


def get_chat_name_from_first_question(chat_message: ChatMessage):
    # Step 1: Get the summary of the first question
    # first_question_summary = summarize_as_title(chat_message.question)
    # Step 2: Process this summary to create a chat name by selecting the first three words
    chat_name = " ".join(chat_message.question.split()[:3])

    return chat_name

1. Chat 

In [ ]:
from dataclasses import asdict, dataclass


@dataclass
class Chat:
    chat_id: str
    user_id: str
    creation_time: str
    chat_name: str

    def __init__(self, chat_dict: dict):
        self.chat_id = chat_dict.get(
            "chat_id"
        )  # pyright: ignore reportPrivateUsage=none
        self.user_id = chat_dict.get(
            "user_id"
        )  # pyright: ignore reportPrivateUsage=none
        self.creation_time = chat_dict.get(
            "creation_time"
        )  # pyright: ignore reportPrivateUsage=none
        self.chat_name = chat_dict.get(
            "chat_name"
        )  # pyright: ignore reportPrivateUsage=none


@dataclass
class ChatHistory:
    chat_id: str
    message_id: str
    user_message: str
    assistant: str
    message_time: str

    def __init__(self, chat_dict: dict):
        self.chat_id = chat_dict.get(
            "chat_id"
        )  # pyright: ignore reportPrivateUsage=none
        self.message_id = chat_dict.get(
            "message_id"
        )  # pyright: ignore reportPrivateUsage=none
        self.user_message = chat_dict.get(
            "user_message"
        )  # pyright: ignore reportPrivateUsage=none
        self.assistant = chat_dict.get(
            "assistant"
        )  # pyright: ignore reportPrivateUsage=none
        self.message_time = chat_dict.get(
            "message_time"
        )  # pyright: ignore reportPrivateUsage=none

    def to_dict(self):
        return asdict(self)

2. Chats 

In [ ]:
from typing import List, Optional, Tuple
from uuid import UUID

from pydantic import BaseModel


class ChatMessage(BaseModel):
    model: str = "gpt-3.5-turbo-16k"
    question: str
    # A list of tuples where each tuple is (speaker, text)
    history: List[Tuple[str, str]]
    temperature: float = 0.0
    max_tokens: int = 256
    use_summarization: bool = False
    chat_id: Optional[UUID] = None
    chat_name: Optional[str] = None


class ChatQuestion(BaseModel):
    model: str = "gpt-3.5-turbo-0613"
    question: str
    temperature: float = 0.0
    max_tokens: int = 256

3. Initialize Chat

In [ ]:
from dataclasses import dataclass
from uuid import UUID

from logger import get_logger
from models.chat import Chat
from models.settings import common_dependencies

logger = get_logger(__name__)


@dataclass
class CreateChatProperties:
    name: str

    def __init__(self, name: str):
        self.name = name


def create_chat(user_id: UUID, chat_data: CreateChatProperties) -> Chat:
    commons = common_dependencies()

    # Chat is created upon the user's first question asked
    logger.info(f"New chat entry in chats table for user {user_id}")

    # Insert a new row into the chats table
    new_chat = {
        "user_id": str(user_id),
        "chat_name": chat_data.name,
    }
    insert_response = commons["supabase"].table("chats").insert(new_chat).execute()
    logger.info(f"Insert response {insert_response.data}")

    return insert_response.data[0]

4. Chat history part.1 

In [ ]:
def format_chat_history(history) -> list[tuple[str, str]]:
    """Format the chat history into a list of tuples (human, ai)"""

    return [(chat.user_message, chat.assistant) for chat in history]

5. Chat history part.2

In [ ]:
from typing import List  # For type hinting

from models.chat import ChatHistory
from models.settings import common_dependencies


def get_chat_history(chat_id: str) -> List[ChatHistory]:
    commons = common_dependencies()
    history: List[ChatHistory] = (
        commons["supabase"]
        .from_("chat_history")
        .select("*")
        .filter("chat_id", "eq", chat_id)
        .order("message_time", desc=False)  # Add the ORDER BY clause
        .execute()
    ).data
    if history is None:
        return []
    else:
        return [
            ChatHistory(message)  # pyright: ignore reportPrivateUsage=none
            for message in history
        ]

6. Chat history part.3

In [ ]:
from typing import List  # For type hinting

from fastapi import HTTPException
from models.chat import ChatHistory
from models.settings import common_dependencies


def update_chat_history(chat_id: str, user_message: str, assistant: str) -> ChatHistory:
    commons = common_dependencies()
    response: List[ChatHistory] = (
        commons["supabase"]
        .table("chat_history")
        .insert(
            {
                "chat_id": str(chat_id),
                "user_message": user_message,
                "assistant": assistant,
            }
        )
        .execute()
    ).data
    if len(response) == 0:
        raise HTTPException(
            status_code=500, detail="An exception occurred while updating chat history."
        )
    return ChatHistory(response[0])  # pyright: ignore reportPrivateUsage=none

7. Chat identification

In [ ]:
from models.chat import Chat
from models.settings import common_dependencies


def get_chat_by_id(chat_id: str) -> Chat:
    commons = common_dependencies()

    response = (
        commons["supabase"]
        .from_("chats")
        .select("*")
        .filter("chat_id", "eq", chat_id)
        .execute()
    )
    return Chat(response.data[0])

8. User chats

In [ ]:
from typing import List

from models.chat import Chat
from models.settings import common_dependencies


def get_user_chats(user_id: str) -> List[Chat]:
    commons = common_dependencies()
    response = (
        commons["supabase"]
        .from_("chats")
        .select("chat_id,user_id,creation_time,chat_name")
        .filter("user_id", "eq", user_id)
        .execute()
    )
    chats = [Chat(chat_dict) for chat_dict in response.data]
    return chats

9. Update chat

In [ ]:
from dataclasses import dataclass
from typing import Optional

from logger import get_logger
from models.chat import Chat
from models.settings import common_dependencies

logger = get_logger(__name__)


@dataclass
class ChatUpdatableProperties:
    chat_name: Optional[str] = None

    def __init__(self, chat_name: Optional[str]):
        self.chat_name = chat_name


def update_chat(chat_id, chat_data: ChatUpdatableProperties) -> Chat:
    commons = common_dependencies()

    if not chat_id:
        logger.error("No chat_id provided")
        return  # pyright: ignore reportPrivateUsage=none

    updates = {}

    if chat_data.chat_name is not None:
        updates["chat_name"] = chat_data.chat_name

    updated_chat = None

    if updates:
        updated_chat = (
            commons["supabase"]
            .table("chats")
            .update(updates)
            .match({"chat_id": chat_id})
            .execute()
        ).data[0]
        logger.info(f"Chat {chat_id} updated")
    else:
        logger.info(f"No updates to apply for chat {chat_id}")
    return updated_chat  # pyright: ignore reportPrivateUsage=none

10. Message update

In [ ]:
from logger import get_logger
from models.chat import ChatHistory
from models.settings import common_dependencies

logger = get_logger(__name__)


def update_message_by_id(
    message_id: str,
    user_message: str = None,  # pyright: ignore reportPrivateUsage=none
    assistant: str = None,  # pyright: ignore reportPrivateUsage=none
) -> ChatHistory:
    commons = common_dependencies()

    if not message_id:
        logger.error("No message_id provided")
        return  # pyright: ignore reportPrivateUsage=none

    updates = {}

    if user_message is not None:
        updates["user_message"] = user_message

    if assistant is not None:
        updates["assistant"] = assistant

    updated_message = None

    if updates:
        updated_message = (
            commons["supabase"]
            .table("chat_history")
            .update(updates)
            .match({"message_id": message_id})
            .execute()
        ).data[0]
        logger.info(f"Message {message_id} updated")
    else:
        logger.info(f"No updates to apply for message {message_id}")
    return ChatHistory(updated_message)  # pyright: ignore reportPrivateUsage=none

11. Logging 

In [ ]:
import logging


def get_logger(logger_name, log_level=logging.INFO):
    logger = logging.getLogger(logger_name)
    logger.setLevel(log_level)
    logger.propagate = False  # Prevent log propagation to avoid double logging

    formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s")

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)

    if not logger.handlers:
        logger.addHandler(console_handler)

    return logger